In [1]:
# Libraries in the use
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import date
import seaborn as sns

### Loading the Data

In [2]:
# US ETFs
US_SECTORS = [
  "XLK",   # Computer & Technology – Tech Select Sector SPDR
  "XLV",   # Medical – Health Care Select Sector SPDR
  "XLE",   # Oils/Energy – Energy Select Sector SPDR
  "XRT",   # Retail/Wholesale – SPDR S&P Retail (broad/equal-weight retail)
  "XLP",   # Consumer Staples – Consumer Staples Select Sector SPDR
  "IGV",   # Business Services – iShares Expanded Tech-Software (IT services/outsourcing proxy)
  "XLB",   # Basic Materials – Materials Select Sector SPDR
  "XLU",   # Utilities – Utilities Select Sector SPDR
  "ITA",   # Aerospace – iShares U.S. Aerospace & Defense
  "XLI",   # Industrial Products – Industrials Select Sector SPDR
  "XLY",   # Consumer Discretionary – Consumer Discretionary Select Sector SPDR
  "ITB",   # Construction – iShares U.S. Home Construction
  "IYT",   # Transportation – iShares U.S. Transportation
  "CARZ"   # Auto/Tires/Trucks – First Trust NASDAQ Global Auto (autos proxy)
]
# Japan ETFs 
JP_SECTORS = [
  "1625.T", # Computer & Technology – Electric Appliances & Precision Instruments (tech hardware proxy)
  "1621.T", # Medical – Pharmaceutical (narrower than US health care)
  "1618.T", # Oils/Energy – Energy Resources
  "1629.T", # Retail/Wholesale – Commercial & Wholesale Trade
  "1617.T", # Consumer Staples – Foods (closest to staples)
  "1626.T", # Business Services – IT & Services, Others
  "1620.T", # Basic Materials – Raw Materials & Chemicals
  "1627.T", # Utilities – Electric Power & Gas
  "1624.T", # Aerospace – Machinery (closest domestic proxy for aerospace/defense supply chain)
  "1623.T", # Industrial Products – Steel & Nonferrous Metals (core industrial inputs)
  "1630.T", # Consumer Discretionary – Retail Trade (broad JP discretionary exposure)
  "1619.T", # Construction – Construction & Materials
  "1628.T", # Transportation – Transportation & Logistics
  "1622.T"  # Auto/Tires/Trucks – Automobiles & Transportation Equipment
]

In [ ]:
# Benchmarks
US_BENCH = "SPY"      # S&P 500 ETF
JP_BENCH = "1321.T"   # Nikkei 225 ETF

start = "2019-01-01"
end = "2022-12-31"

# Loading the data
def load_sector_data(tickers, start, end):
    """Download sector data and clean column names"""
    df = yf.download(tickers, start=start, end=end, auto_adjust=False, progress=False)["Adj Close"]
    df = df.reset_index()  # Move date from index to column
    df.columns.name = None  # Remove multi-index name
    return df

us_data = load_sector_data(US_SECTORS, start, end)
jp_data = load_sector_data(JP_SECTORS, start, end)

### Computing Returns in each phase 

<div align="left">
Using the following formulas:

$$ R_{crash} = \frac{P_{trough}}{P_{peak}} - 1$$
$$ R_{recovery} = \frac{P_{recovery}}{P_{trough}} - 1$$
$$ R_{post} = \frac{P_{end}}{P_{recovery}} - 1$$


</div>

1. Computing the Return

In [ ]:
def compute_phase_return(df, phase):
    """
    Compute returns of each ETFs during: Crash, Recovery, and Post-Covid.

    Parameters: 
        df: pd.DataFrame - our data of the ETFs (us_data, jp_data)

        phase: Dates from the market_phases.csv

    Output:
        pd.DataFrame - Summary Table of returns (%)
    """

    peak = phase["peak_date"]
    trough = phase["trough_date"]
    recovery = phase["recovery_date"]

    df = df.